In [10]:
from pathlib import Path
import sys
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
%load_ext autoreload

In [11]:
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam, RMSprop



%matplotlib inline
dir_package = str(Path('./facebias').resolve())
if dir_package not in sys.path:
    print(f"adding {dir_package} to path")
    sys.path.append(dir_package)
else:
    print(dir_package)
from facebias.models.mlp import get_finetuned_mlp
from facebias.iotools import split_bfw_features

/Users/jrobby/GitHub/facerec-bias-bfw/code/facebias


In [12]:
font = {'font.family': 'serif',
        'font.serif' : 'Times New Roman',
        'font.color':  'darkred',
        'font.weight': 'normal',
        'font.size': 16,
        }
sns.set(style="white", rc={"axes.facecolor": (0, 0, 0, 0)})
sns.set_style('whitegrid', font)

In [ ]:
d_input='../data/bfw/features/arcface/features.pkl'
f_meta='../data/bfw/meta/bfw-fold-meta-lut.csv'
batch_size=64
do_gender=True
do_ethnicity=False
d_weights = 'train_models'

f_gender_weights = f"{d_weights}/gender_fc_model.h5"
f_ethnicity_weights = f"{d_weights}/ethnicity_fc_model.h5"


dir_features = str(Path(d_input).parent)

In [17]:
print(f"Gender")
ref_tr, features_tr, labels_tr, ref_val, features_val, labels_val \
    = split_bfw_features(f_meta, dir_features, val_fold=5)

In [21]:
model = get_finetuned_mlp(features_tr.shape[1:], optimizer='adam')
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 2048)              1050624   
_________________________________________________________________
dropout_6 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dropout_7 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 512)               524800    
_________________________________________________________________
dropout_8 (Dropout)          (None, 512)              

In [22]:
val_fold=5
# model.fit(
#     features_tr,
#     labels_tr,
#     batch_size=batch_size,
#     validation_data=(features_val, labels_val),
# )
path_in = path_weights / str(val_fold) / "gender_fc_model.h5"
# path_in=
model.load_weights(str(path_in))

Train on 16000 samples, validate on 4000 samples
16000/16000 [==============================] - 17s 1ms/sample - loss: 0.2061 - accuracy: 0.9087 - val_loss: 0.2085 - val_accuracy: 0.9227


ValueError: Shapes (512, 2048) and (512, 512) are incompatible

In [ ]:
if do_ethnicity:
    for val_fold in range(1, N_FOLDS + 1):
        ref_tr, features_tr, labels_tr, ref_val, features_val, labels_val \
            = split_bfw_features(f_meta, dir_features, 'ethnicity')

        opt = Adam(lr=1e-4) if args.optimizer == 'adam' \
            else RMSprop(0.0001, decay=1e-6)
        model = get_finetuned_mlp(features_tr.shape[1:], optimizer=opt,
                                  loss='sparse_categorical_crossentropy',
                                  output_activation='softmax', output_size=4)

        model.fit(
            features_tr,
            labels_tr,
            epochs=epochs,
            batch_size=batch_size,
            validation_data=(features_val, labels_val),
            callbacks=[tb_callback]
        )
        path_in = path_weights / str(val_fold) / "ethnicity_fc_model.h5"
        model.save_weights(str(path_in))
